## Loading Data 

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test_nolabel.csv')
val_df = pd.read_csv('data/val.csv')

In [56]:
print(train_df.head())
print(train_df.info())

     id  label                                               text
0  8901      5  Bennett 's naturalistic performance speaks vol...
1  2506      5  Shot in rich , shadowy black-and-white , Devil...
2  2381      5  More than their unique residences , Home Movie...
3  1262      3  The movie should be credited with remembering ...
4  2542      4  Audiences are advised to sit near the back and...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8298 entries, 0 to 8297
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8298 non-null   int64 
 1   label   8298 non-null   int64 
 2   text    8298 non-null   object
dtypes: int64(2), object(1)
memory usage: 194.6+ KB
None


### Check class Imbalance

In [57]:
print(train_df["label"].value_counts(normalize=True))
print(val_df["label"].value_counts(normalize=True))


label
2    0.263919
4    0.261388
3    0.192818
5    0.156905
1    0.124970
Name: proportion, dtype: float64
label
4    0.275028
2    0.257593
3    0.173791
5    0.151856
1    0.141732
Name: proportion, dtype: float64


## Preprocess

In [58]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer


def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])  # Remove stopwords
    return text




### Apply text cleaning

In [59]:
train_df["clean_text"] = train_df["text"].apply(preprocess_text)
val_df["clean_text"] = val_df["text"].apply(preprocess_text)
test_df["clean_text"] = test_df["text"].apply(preprocess_text)

## Convert Text to Numerical Features (TF-IDF

In [60]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

#### Transform the text data

In [61]:
X_train = vectorizer.fit_transform(train_df["clean_text"]).toarray()
X_val = vectorizer.transform(val_df["clean_text"]).toarray()
X_test = vectorizer.transform(test_df["clean_text"]).toarray()

In [62]:
y_train = train_df["label"].values
y_val = val_df["label"].values

## Build the MLP Model Using Sequential

In [63]:
model = Sequential([
    Dense(512, activation="relu", input_shape=(X_train.shape[1],)),  # First hidden layer
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(256, activation="relu"),  # Second hidden layer
    Dropout(0.3),
    Dense(128, activation="relu"),  # Third hidden layer
    Dense(1, activation="sigmoid")  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])


C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Train the Model

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,  # Increase if needed
    batch_size=32
)


Epoch 1/10
143/260 ━━━━━━━━━━━━━━━━━━━━ 20s 173ms/step - accuracy: 0.1336 - loss: -1834.9185

## Evaluate the Model

In [52]:
# Predict on validation set
y_val_pred = (model.predict(X_val) > 0.5).astype(int)

# Accuracy score
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Classification Report
print(classification_report(y_val, y_val_pred))


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Validation Accuracy: 0.1417
              precision    recall  f1-score   support

           1       0.14      1.00      0.25       252
           2       0.00      0.00      0.00       458
           3       0.00      0.00      0.00       309
           4       0.00      0.00      0.00       489
           5       0.00      0.00      0.00       270

    accuracy                           0.14      1778
   macro avg       0.03      0.20      0.05      1778
weighted avg       0.02      0.14      0.04      1778



C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Make Predictions on the Test Set

In [53]:
y_test_pred = (model.predict(X_test) > 0.5).astype(int)


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [54]:
submission = pd.DataFrame({"id": test_df["id"], "label": y_test_pred.flatten()})
submission.to_csv("submission.csv", index=False)
